# Multi-channel ANN
## Number of channels: 2

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import trange
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import trange

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# torch.manual_seed(2021)

In [ ]:
def rename_column(data):
    return data.rename(columns={'평균 수온(°C)':'temperature', '일시':'date'}, inplace=True)

years = '2016-2021-06-15'
df = pd.read_csv('./data/' + years + '_all.csv')
df['일시'] = df['Unnamed: 0']
df = df.drop(columns='Unnamed: 0')
df['일시'] = pd.to_datetime(df['일시'])
rename_column(df)
df.set_index('date',inplace=True)
data = df
display(df.head())

result = []     # metric 담을 list 생성

In [ ]:
window_size = 180
data['rolling_mean'] = data.rolling(window_size).mean()
rename_column(data)
data

In [ ]:
data = data.dropna()
data.plot()

In [ ]:
class TimeSeriesDatasetMulti(Dataset):
  def __init__(self, ts:np.array, lookback_size:int, forecast_size:int):
    self.lookback_size = lookback_size
    self.forecast_size = forecast_size
    self.data = ts

  def __len__(self):
    return len(self.data) - self.lookback_size - self.forecast_size + 1

  def __getitem__(self, i):
    idx = (i + self.lookback_size)
    look_back = self.data[i:idx]
    forecast = self.data[idx:idx + self.forecast_size]
    return look_back, forecast
  
class NetMulti(nn.Module):
  def __init__(self, d_in, d_out, d_hidden1, d_hidden2, c_in, activation=F.relu):
    super().__init__()
    self.lin1 = nn.Linear(d_in*c_in, d_hidden1)
    self.lin2 = nn.Linear(d_hidden1, d_hidden2)
    self.lin3 = nn.Linear(d_hidden2, d_out*c_in)
    self.activation = activation
    self.c_in = c_in
    self.d_out = d_out

  def forward(self, x):
    x = x.flatten(1)
    x = self.lin1(x)
    x = self.activation(x)
    x = self.lin2(x)
    x = self.activation(x)
    x = self.lin3(x).reshape(-1, self.d_out, self.c_in)
    return x

def mape(y_pred, y_true):
  return (np.abs(y_pred - y_true)/y_true).mean() * 100

def mae(y_pred, y_true):
  return np.abs(y_pred - y_true).mean()

def mse(y_pred, y_true):
  return np.mean((np.array(y_true) - np.array(y_pred)) ** 2)

def rmse(y_pred, y_true):
  return np.sqrt(mse(y_pred, y_true))

def r_squared(y_pred, y_true):
  return r2_score(y_true, y_pred)

def r2_score(y_pred, y_true):
  ss_tot = np.sum((y_true - np.mean(y_true))**2)
  ss_res = np.sum((y_true - y_pred)**2)
  r2 = 1 - (ss_res / ss_tot)
  return r2

In [ ]:
def generate_time_series_loaders(pred_size, lookback_size, forecast_size):
    trn_df = data[:-pred_size].to_numpy(dtype=np.float32)
    tst_df = data[-pred_size - lookback_size:].to_numpy(dtype=np.float32)
    tst_y = tst_df[-pred_size:]

    # non-scaling
    trn_Ods = TimeSeriesDatasetMulti(trn_df, lookback_size, forecast_size)
    tst_Ods = TimeSeriesDatasetMulti(tst_df, lookback_size, forecast_size)

    trn_Odl = DataLoader(trn_Ods, batch_size=32, shuffle=True)
    tst_Odl = DataLoader(tst_Ods, batch_size=pred_size, shuffle=False)

    # scaling
    scaler = MinMaxScaler()
    trn_scaled = scaler.fit_transform(data[:-pred_size].to_numpy(dtype=np.float32))
    tst_scaled = scaler.transform(data[-pred_size-lookback_size:].to_numpy(dtype=np.float32))

    trn_ds = TimeSeriesDatasetMulti(trn_scaled, lookback_size, forecast_size)

    trn_dl = DataLoader(trn_ds, batch_size=32, shuffle=True)

    return tst_y, trn_Ods, tst_Ods, trn_Odl, tst_Odl, trn_ds, trn_dl

# Hyper Parameter
### pred_size = 15, 30, 60, 90

In [ ]:
'''
order_list = [
    [15, 5, 1, 512, 128, 2, 200, 0.0001], [15, 5, 1, 512, 128, 2, 400, 0.0001], [15, 5, 1, 512, 128, 2, 850, 0.0001], 
    [15, 5, 5, 512, 128, 2, 200, 0.0001], [15, 5, 5, 512, 128, 2, 400, 0.0001], [15, 5, 5, 512, 128, 2, 850, 0.0001], 
    [15, 5, 15, 512, 128, 2,200, 0.0001], [15, 5, 15, 512, 128, 2, 400, 0.0001], [15, 5, 15, 512, 128, 2, 850, 0.0001], 
    
    [15, 15, 1, 512, 128, 2, 200, 0.0001], [15, 15, 1, 512, 128, 2, 400, 0.0001], [15, 15, 1, 512, 128, 2, 850, 0.0001], 
    [15, 15, 5, 512, 128, 2, 200, 0.0001], [15, 15, 5, 512, 128, 2, 400, 0.0001], [15, 15, 5, 512, 128, 2, 850, 0.0001], 
    [15, 15, 15, 512, 128, 2, 200, 0.0001], [15, 15, 15, 512, 128, 2, 400, 0.0001], [15, 15, 15, 512, 128, 2, 850, 0.0001], 
    
    [15, 30, 1, 512, 128, 2, 200, 0.0001], [15, 30, 1, 512, 128, 2, 400, 0.0001], [15, 30, 1, 512, 128, 2, 850, 0.0001], 
    [15, 30, 5, 512, 128, 2, 200, 0.0001], [15, 30, 5, 512, 128, 2, 400, 0.0001], [15, 30, 5, 512, 128, 2, 850, 0.0001], 
    [15, 30, 15, 512, 128, 2, 200, 0.0001], [15, 30, 15, 512, 128, 2, 400, 0.0001], [15, 30, 15, 512, 128, 2, 850, 0.0001], 
    
    [15, 90, 1, 512, 128, 2, 200, 0.0001], [15, 90, 1, 512, 128, 2, 400, 0.0001], [15, 90, 1, 512, 128, 2, 850, 0.0001], 
    [15, 90, 5, 512, 128, 2, 200, 0.0001], [15, 90, 5, 512, 128, 2, 400, 0.0001], [15, 90, 5, 512, 128, 2, 850, 0.0001], 
    [15, 90, 15, 512, 128, 2, 200, 0.0001], [15, 90, 15, 512, 128, 2, 400, 0.0001], [15, 90, 15, 512, 128, 2, 850, 0.0001], 
    
    [30, 5, 1, 512, 128, 2, 200, 0.0001], [30, 5, 1, 512, 128, 2, 400, 0.0001], [30, 5, 1, 512, 128, 2, 850, 0.0001], 
    [30, 5, 5, 512, 128, 2, 200, 0.0001], [30, 5, 5, 512, 128, 2, 400, 0.0001], [30, 5, 5, 512, 128, 2, 850, 0.0001], 
    [30, 5, 15, 512, 128, 2, 200, 0.0001], [30, 5, 15, 512, 128, 2, 400, 0.0001], [30, 5, 15, 512, 128, 2, 850, 0.0001], 
    
    [30, 15, 1, 512, 128, 2, 200, 0.0001], [30, 15, 1, 512, 128, 2, 400, 0.0001], [30, 15, 1, 512, 128, 2, 850, 0.0001], 
    [30, 15, 5, 512, 128, 2, 200, 0.0001], [30, 15, 5, 512, 128, 2, 400, 0.0001], [30, 15, 5, 512, 128, 2, 850, 0.0001], 
    [30, 15, 15, 512, 128, 2, 200, 0.0001], [30, 15, 15, 512, 128, 2, 400, 0.0001], [30, 15, 15, 512, 128, 2, 850, 0.0001], 
    
    [30, 30, 1, 512, 128, 2, 200, 0.0001], [30, 30, 1, 512, 128, 2, 400, 0.0001], [30, 30, 1, 512, 128, 2, 850, 0.0001], 
    [30, 30, 5, 512, 128, 2, 200, 0.0001], [30, 30, 5, 512, 128, 2, 400, 0.0001], [30, 30, 5, 512, 128, 2, 850, 0.0001], 
    [30, 30, 15, 512, 128, 2, 200, 0.0001], [30, 30, 15, 512, 128, 2, 400, 0.0001], [30, 30, 15, 512, 128, 2, 850, 0.0001], 
    
    [30, 90, 1, 512, 128, 2, 200, 0.0001], [30, 90, 1, 512, 128, 2, 400, 0.0001], [30, 90, 1, 512, 128, 2, 850, 0.0001], 
    [30, 90, 5, 512, 128, 2, 200, 0.0001], [30, 90, 5, 512, 128, 2, 400, 0.0001], [30, 90, 5, 512, 128, 2, 850, 0.0001], 
    [30, 90, 15, 512, 128, 2, 200, 0.0001], [30, 90, 15, 512, 128, 2, 400, 0.0001], [30, 90, 15, 512, 128, 2, 850, 0.0001],

    [60, 5, 1, 512, 128, 2, 200, 0.0001], [60, 5, 1, 512, 128, 2, 400, 0.0001], [60, 5, 1, 512, 128, 2, 850, 0.0001], 
    [60, 5, 5, 512, 128, 2, 200, 0.0001], [60, 5, 5, 512, 128, 2, 400, 0.0001], [60, 5, 5, 512, 128, 2, 850, 0.0001], 
    [60, 5, 15, 512, 128, 2, 200, 0.0001], [60, 5, 15, 512, 128, 2, 400, 0.0001], [60, 5, 15, 512, 128, 2, 850, 0.0001], 
    
    [60, 15, 1, 512, 128, 2, 200, 0.0001], [60, 15, 1, 512, 128, 2, 400, 0.0001], [60, 15, 1, 512, 128, 2, 850, 0.0001], 
    [60, 15, 5, 512, 128, 2, 200, 0.0001], [60, 15, 5, 512, 128, 2, 400, 0.0001], [60, 15, 5, 512, 128, 2, 850, 0.0001], 
    [60, 15, 15, 512, 128, 2, 200, 0.0001], [60, 15, 15, 512, 128, 2, 400, 0.0001], [60, 15, 15, 512, 128, 2, 850, 0.0001], 
    
    [60, 30, 1, 512, 128, 2, 200, 0.0001], [60, 30, 1, 512, 128, 2, 400, 0.0001], [60, 30, 1, 512, 128, 2, 850, 0.0001], 
    [60, 30, 5, 512, 128, 2, 200, 0.0001], [60, 30, 5, 512, 128, 2, 400, 0.0001], [60, 30, 5, 512, 128, 2, 850, 0.0001], 
    [60, 30, 15, 512, 128, 2, 200, 0.0001], [60, 30, 15, 512, 128, 2, 400, 0.0001], [60, 30, 15, 512, 128, 2, 850, 0.0001], 
    
    [60, 90, 1, 512, 128, 2, 200, 0.0001], [60, 90, 1, 512, 128, 2, 400, 0.0001], [60, 90, 1, 512, 128, 2, 850, 0.0001], 
    [60, 90, 5, 512, 128, 2, 200, 0.0001], [60, 90, 5, 512, 128, 2, 400, 0.0001], [60, 90, 5, 512, 128, 2, 850, 0.0001], 
    [60, 90, 15, 512, 128, 2, 200, 0.0001], [60, 90, 15, 512, 128, 2, 400, 0.0001], [60, 90, 15, 512, 128, 2, 850, 0.0001], 
    
    [90, 5, 1, 512, 128, 2, 200, 0.0001], [90, 5, 1, 512, 128, 2, 400, 0.0001], [90, 5, 1, 512, 128, 2, 850, 0.0001], 
    [90, 5, 5, 512, 128, 2, 200, 0.0001], [90, 5, 5, 512, 128, 2, 400, 0.0001], [90, 5, 5, 512, 128, 2, 850, 0.0001], 
    [90, 5, 15, 512, 128, 2, 200, 0.0001], [90, 5, 15, 512, 128, 2, 400, 0.0001], [90, 5, 15, 512, 128, 2, 850, 0.0001], 
    
    [90, 15, 1, 512, 128, 2, 200, 0.0001], [90, 15, 1, 512, 128, 2, 400, 0.0001], [90, 15, 1, 512, 128, 2, 850, 0.0001], 
    [90, 15, 5, 512, 128, 2, 200, 0.0001], [90, 15, 5, 512, 128, 2, 400, 0.0001], [90, 15, 5, 512, 128, 2, 850, 0.0001], 
    [90, 15, 15, 512, 128, 2, 200, 0.0001], [90, 15, 15, 512, 128, 2, 400, 0.0001], [90, 15, 15, 512, 128, 2, 850, 0.0001], 
    
    [90, 30, 1, 512, 128, 2, 200, 0.0001], [90, 30, 1, 512, 128, 2, 400, 0.0001], [90, 30, 1, 512, 128, 2, 850, 0.0001], 
    [90, 30, 5, 512, 128, 2, 200, 0.0001], [90, 30, 5, 512, 128, 2, 400, 0.0001], [90, 30, 5, 512, 128, 2, 850, 0.0001], 
    [90, 30, 15, 512, 128, 2, 200, 0.0001], [90, 30, 15, 512, 128, 2, 400, 0.0001],[90, 30, 15, 512, 128, 2, 850, 0.0001], 
    
    [90, 90, 1, 512, 128, 2, 200, 0.0001], [90, 90, 1, 512, 128, 2, 400, 0.0001], [90, 90, 1, 512, 128, 2, 850, 0.0001], 
    [90, 90, 5, 512, 128, 2, 200, 0.0001], [90, 90, 5, 512, 128, 2, 400, 0.0001], [90, 90, 5, 512, 128, 2, 850, 0.0001], 
    [90, 90, 15, 512, 128, 2, 200, 0.0001], [90, 90, 15, 512, 128, 2, 400, 0.0001], [90, 90, 15, 512, 128, 2, 850, 0.0001]
]
'''

# Model Fitting and Forecasting

In [ ]:
order_list = [[15, 5, 1, 512, 128, 2, 850, 0.0001], [30, 15, 1, 512, 128, 2, 850, 0.0001], [60, 30, 1, 512, 128, 2, 850, 0.0001], [90, 90, 1, 512, 128, 2, 850, 0.0001]]
cnt = 1

for pred_size, lookback_size, forecast_size, hidden_dim1, hidden_dim2, channel_size, epoch, lr in order_list:
	print(f'pred_size: {pred_size}, lookback_size: {lookback_size}, forecast_size: {forecast_size}, hidden_dim1: {hidden_dim1}, hidden_dim2: {hidden_dim2}, channel_size: {channel_size}, epoch: {epoch}, lr: {lr}')

	tst_y, trn_Ods, tst_Ods, trn_Odl, tst_Odl, trn_ds, trn_dl = generate_time_series_loaders(pred_size, lookback_size, forecast_size)

	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	
	net = NetMulti(lookback_size, forecast_size, hidden_dim1, hidden_dim2, channel_size)
	net.to(device)

	optim = torch.optim.AdamW(net.parameters(), lr=lr)
	pbar = trange(epoch)
	
	losses = []
	trn_losses = []
	tst_rmse_losses = []

	for i in pbar:
		net.train()
		trn_loss = .0

		for x, y in trn_Odl:
			x, y = x.to(device), y.to(device)
			p = net(x)
			optim.zero_grad()
			loss = F.mse_loss(p, y)
			loss.backward()
			optim.step()
			trn_loss += loss.item() * len(y)
		trn_loss = trn_loss / len(trn_Ods)

		net.eval()
		with torch.inference_mode():
			x, y = next(iter(tst_Odl))
			x, y = x.to(device), y.to(device)
			p = net(x)
			tst_loss = F.mse_loss(p, y) 
			y = y.cpu()
			p = p.cpu()    
			tst_mape = mape(p, y)
			tst_mae = mae(p, y)
			tst_rmse = rmse(p, y)
			tst_mse = mse(p, y)

		pbar.set_postfix({
			'Multi-channel trn_loss':trn_loss, 'tst_loss':tst_loss.item(), 'tst_mape':tst_mape.item(), 
			'tst_mae':tst_mae.item(), 'tst_rmse':tst_rmse.item()
		})
		tst_loss = tst_loss.cpu()  
		losses.append(tst_loss)
		trn_losses.append(trn_loss)
		tst_rmse_losses.append(tst_rmse)

	path = f'./NN_{channel_size}ch/model/model_ANN_{channel_size}multi_{cnt}.pth'
	torch.save(net.state_dict(), path)

	plot_start = 50     # 그래프 디테일하게 보기 위해 확대
	epochs_to_plot = range(plot_start, epoch)
	plt.figure(figsize=(10, 5))
	plt.title(f"Neural Network {channel_size}Multi-channel_({pred_size},{lookback_size},{forecast_size})_{epoch}_{lr}")
	# plt.plot(epochs_to_plot, losses[plot_start:], label='tst_loss')	# 실제 예측 시 사용안함. 필요여부에 따라 사용가능
	plt.plot(epochs_to_plot, tst_rmse_losses[plot_start:], label='tst_rmse')
	plt.plot(epochs_to_plot, trn_losses[plot_start:], label='train_loss')
	plt.xticks(range(plot_start, epoch, 100))
	plt.legend()
	plt.savefig(f'./NN_{channel_size}ch/fig/ANN_{channel_size}multi_loss_{years}_({pred_size},{lookback_size},{forecast_size})_{epoch}_{lr}_{cnt}.png')
	plt.show

	preds = []
	x, y = trn_Ods[len(trn_Ods)-1]
	
	net = NetMulti(lookback_size, forecast_size, hidden_dim1, hidden_dim2, channel_size)
	net.load_state_dict(torch.load(path))

	net.eval()
	for _ in range(int(pred_size/forecast_size)):
		if y.shape == (1, forecast_size, channel_size):
			y = y.squeeze(0)
		x = np.concatenate([x, y])[-lookback_size:]
		x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
		x_tensor.to(device)

		with torch.inference_mode():
			y = net(x_tensor).cpu()
		preds.append(y)

	preds = np.concatenate(preds)
	final_preds = preds[:,:,0].flatten()
	tst_y = tst_y[:,0]

	MAPE = mape(final_preds, tst_y)
	MAE = mae(final_preds, tst_y)
	MSE = mse(final_preds, tst_y)
	RMSE = rmse(final_preds, tst_y)
	R2 = r_squared(final_preds, tst_y)
	R2S = r2_score(final_preds, tst_y)
	result.append([(pred_size, lookback_size, forecast_size, epoch, lr), MAPE, MAE, MSE, RMSE, R2, R2S])

	plt.figure(figsize=(10, 5))
	plt.title(f"NN {channel_size}Multi-channel_({pred_size},{lookback_size},{forecast_size})_{epoch}_MAPE:{MAPE:.3f}, MAE:{MAE:.3f}, MSE:{MSE:.3f}, RMSE:{RMSE:.3f}, R2:{R2:.3f}, R2S:{R2S:.3f}")
	plt.plot(range(pred_size), tst_y, label="True")
	plt.plot(range(pred_size), final_preds, label="Prediction")
	plt.legend()
	plt.savefig(f'./NN_{channel_size}ch/result/ANN_{channel_size}multi_{years}_({pred_size},{lookback_size},{forecast_size})_{epoch}_{lr}_{cnt}.png')
	plt.show()

	cnt += 1

# 결과 DataFrame 생성

In [ ]:
pd.set_option('float_format', '{:.4f}'.format)
result_df = pd.DataFrame(result, columns=['order', 'MAPE', 'MAE', 'MSE', 'RMSE', 'R2', 'R2S'])
result_df.set_index('order', inplace=True)
result_df.to_csv(f'./NN_{channel_size}ch/ANN_' + str(channel_size) + 'multi_result_' + years + '.csv')
result_df